In [ ]:
import streamlit as st
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline as hf_pipeline
from transformers import AutoTokenizer
from pathlib import Path
import joblib

st.title("Intelligent Customer Feedback Analysis")

uploaded_file = st.file_uploader("Upload CSV with column 'feedback'", type=["csv"])
if uploaded_file:
    df = pd.read_csv(uploaded_file)
    st.write("Preview:")
    st.dataframe(df.head())
    if st.button("Analyze"):
        with st.spinner("Loading models..."):
            sentiment = hf_pipeline("sentiment-analysis", model="distilbert-base-uncased", tokenizer="distilbert-base-uncased")
            summarizer = hf_pipeline("summarization", model="facebook/bart-large-cnn")
        st.success("Models loaded")
        results = []
        for idx, row in df.head(40).iterrows():
            text = str(row['feedback'])
            s = sentiment(text)[0]
            sum_short = summarizer(text, max_length=25, min_length=8, do_sample=False)[0]['summary_text']
            results.append({"feedback": text, "label": s['label'], "score": s['score'], "summary": sum_short})

        st.write(pd.DataFrame(results))
